# Part 2 - WML Federated Learning with MNIST for Party 

### Learning Goals

When you complete the Part 2 - WML Federated Learning with MNIST for Party, you should know how to:

- Load the data that you intend to use in the Federated Learning experiment.
- Install IBM Federated Learning libraries.
- Define a data handler. For more details on data handlers, see <a href = "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-cus-dh.html?audience=wdp&context=cpdaas" target="_blank" rel="noopener no referrer">Customizing the data handler</a>.
- Configure the party to train data with the aggregator.

<div class="alert alert-block alert-info">This notebook is intended to be run by the administrator or connecting party of the Federated Learning experiment.
</div>

## Table of Contents

1. [Load the data](#load)<br>
2. [Install Federated Learning libraries](#install)<br>
3. [Define a Data Handler](#data-handler)<br>
4. [Configure the party](#config)<br>
5. [Train with Federated Learning](#train)<br>
6. [Summary](#summary)

<div class="alert alert-block alert-warning">Before you run this notebook, you must have already run Part 1 - WML Federated Learning with MNIST for Admin. If you have not, open the notebook and run through that notebook first.
</div>

### Check Data Files Exist

In this part, you need to upload the datasets that will be used by the party to train the Federated Learning model with.

For this tutorial, the datasets are supplied as part of the project. We will be using the MNIST datasets in our example. 

Simply verify that the packages exist by running the bash code as follows:

In [1]:
!ls /project_data/data_asset

mnist-keras-test.pkl  mnist-keras-train.pkl


### Paste Variables From Aggregator Notebook

Paste in the ID credentials you got from the end of the Part 1 notebook. If you have not run through Part 1, open the notebook and run through it first.

In [2]:
CP4D_HOST = ""
CP4D_URL = ""
WS_USER = "admin"
WS_PASSWORD = "password"
TRAINING_ID = ""
RTS_ID = ""

<a id = "install"></a>
## 2. Install Federated Learning libraries

In this section, we will install the necessary libraries and other packages to call for Federated Learning with the Python client.

### Install the IBM WML SDK with FL

This installs the IBM Watson Machine Learning CLI along with the whole software development package with Federated Learning.

In [3]:
!pip install --upgrade ibm-watson-machine-learning

Requirement already up-to-date: ibm-watson-machine-learning in /opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages (1.0.44)


<a id = "2.2"></a>
### 2.2 Install the libraries

In [4]:
!pip install environs parse websockets jsonpickle pandas pytest pyYAML requests pathlib2 psutil setproctitle tabulate lz4 opencv-python gym ray==0.8.0 cloudpickle==1.3.0 image

<a id = "2.3"></a>
### 2.3 Install the frameworks

In [5]:
!pip install tensorflow==2.1.0 scikit-learn==0.23.1 keras==2.2.4 numpy==1.17.4 scipy==1.4.1 

<a id = "2.4"></a>
### 2.4 Import the Party

The following code imports the package for the party, and ensures that it is loaded.

In [6]:
import ibmfl.party_env_validator
from ibmfl.party.party import Party

No module named 'scikit-learn' Required version is  0.23.1


Using TensorFlow backend.


No module named 'PyYAML'
No module named 'diffprivlib'
No module named 'opencv-python'


<a id = "data-handler"></a>
## 3. Define a Data Handler

The party should run a data handler to ensure that their datasets are in compatible format and consistent. In this tutorial, an example data handler for the MNIST dataset is provided. 

For more details on data handlers, see <a href = "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-cus-dh.html?audience=wdp&context=cpdaas" target="_blank" rel="noopener no referrer">Customizing the data handler</a>.

In [7]:
%%writefile mnist_keras_data_handler.py
from keras.preprocessing.image import ImageDataGenerator
import logging

import numpy as np
from keras.utils import np_utils

from ibmfl.data.data_handler import DataHandler
from ibmfl.util.datasets import load_mnist

logger = logging.getLogger(__name__)



class MnistTFDataHandler(DataHandler):
    """
       Data handler for MNIST dataset.
       """

    def __init__(self, data_config=None, channels_first=False):
        super().__init__()
        self.file_name = None
        if data_config is not None:
            if 'train_file' in data_config:
                self.train_file_name = data_config['train_file']
            if 'test_file' in data_config:
                self.test_file_name = data_config['test_file']

    def get_data(self, nb_points=500):
        """
        Gets pre-process mnist training and testing data. Because this method
        is for testing it takes as input the number of datapoints, nb_points,
        to be included in the training and testing set.

        :param: nb_points: Number of data points to be included in each set
        :type nb_points: `int`
        :return: training data
        :rtype: `tuple`
        """
        if self.file_name is None:
            (x_train, y_train), (x_test, y_test) = load_mnist()
            # Reduce datapoints to make test faster
            x_train = x_train[:nb_points]
            y_train = y_train[:nb_points]
            x_test = x_test[:nb_points]
            y_test = y_test[:nb_points]
        else:
            try:
                logger.info(
                    'Loaded training data from ' + str(self.file_name))
                data_train = np.load(self.file_name)
                with open("MNIST-pkl/mnist-keras-train.pkl", 'rb') as f:
                    (x_train, y_train)= pickle.load(f)

                with open("MNIST-pkl/mnist-keras-train.pkl", 'rb') as f:
                    (x_test, y_test)= pickle.load(f)
                
            except Exception:
                raise IOError('Unable to load training data from path '
                              'provided in config file: ' +
                              self.file_name)

        # Add a channels dimension
        import tensorflow as tf
        x_train = x_train[..., tf.newaxis]
        x_test = x_test[..., tf.newaxis]

        print('x_train shape:', x_train.shape)
        print(x_train.shape[0], 'train samples')
        print(x_test.shape[0], 'test samples')

        return (x_train, y_train), (x_test, y_test)

Overwriting mnist_keras_data_handler.py


### Verify Data Handler Exists

In [8]:
!ls -al

total 21200
drwxr-x---. 5 wsuser  watsonstudio      155 Dec  8 20:23 .
drwxrwx---. 1 wsbuild wsbuild            88 Dec  8 20:23 ..
drwxr-x---. 2 wsuser  watsonstudio        6 Dec  8 20:23 assets
-rw-r-----. 1 wsuser  watsonstudio     2496 Dec  8 20:52 mnist_keras_data_handler.py
-rw-r-----. 1 wsuser  watsonstudio 11490434 Dec  8 20:23 mnist.npz
-rw-r-----. 1 wsuser  watsonstudio 10161039 Dec  8 20:23 model_package.zip
drwxr-x---. 2 wsuser  watsonstudio       53 Dec  8 20:23 __pycache__
-rw-r-----. 1 wsuser  watsonstudio    48079 Dec  8 20:23 saved_model.pb
drwxr-x---. 2 wsuser  watsonstudio       66 Dec  8 20:23 variables


<a id = "config"></a>
## 4. Configure the party

Each party must run their party configuration file to call out to the aggregator. Here is an example of a party configuration.

Because you had already defined the training ID, RTS ID and data handler in the previous sections of this notebook, and the local training and protocol handler are all defined by the SDK, you will only need to define the information for the dataset file under `["data"]["info"]`. 

In this tutorial, the data path is already defined as we have loaded the examplar MNIST dataset from previous sections.

In [9]:
from pathlib import Path
working_dir = !pwd
pwd = working_dir[0]

party_config = {
  "aggregator": {
    "ip": CP4D_HOST  + "/ml/v4/trainings/" + TRAINING_ID
  },
  "connection": {
    "info": {
      "id": RTS_ID,
    }
  },
  "data": {
    "info": {
      "train_file": "/project_data/data_asset/mnist-keras-train.pkl",
      "test_file": "/project_data/data_asset/mnist-keras-test.pkl"
    },
    "name": "MnistTFDataHandler",
    "path": pwd + "/mnist_keras_data_handler.py"
  },
  "local_training": {
    "name": "LocalTrainingHandler",
    "path": "ibmfl.party.training.local_training_handler"
  },
  "protocol_handler": {
    "name": "PartyProtocolHandler",
    "path": "ibmfl.party.party_protocol_handler"
  }
}

In [ ]:
print(party_config)

<a id = "train"></a>
## 5. Connect and train with Federated Learning

Here you can finally connect to the aggregator to begin training.

#### Obtain Cloud Authentication Token

In [11]:
wml_credentials = {
  "username": WS_USER,
  "password": WS_PASSWORD,
  "instance_id" : "wml_local",
  "url": "https://" + CP4D_HOST,
  "version": "3.5"
}

from ibm_watson_machine_learning import APIClient
wml_client = APIClient(wml_credentials)
auth_token = "Bearer " + wml_client.wml_token


2020-12-08 20:53:12,605 - ibmfl.util.config - WARNING - Yaml configuration file not found: log_config.yaml


In [ ]:
print("Token: " + auth_token)

### 5.1 Establish Connection To Aggregator

In [13]:
p = Party( config_dict = party_config, token = auth_token, self_signed_cert=True  )

2020-12-08 20:53:12,615 | 1.0.0 | INFO | ibmfl.util.config                                  | No model config provided for this setup.
2020-12-08 20:53:12,617 | 1.0.0 | INFO | ibmfl.util.config                                  | No fusion config provided for this setup.
2020-12-08 20:53:12,624 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | Websockets Sender initialized
2020-12-08 20:53:12,627 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | WSConnection : Initialize Party Communications
2020-12-08 20:53:12,629 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | **** PartySendLoopThread
2020-12-08 20:53:12,632 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | PartySendLoop: Holding for message to send
2020-12-08 20:53:12,633 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | **** PartyRecvLoopThread
2020-12-08 20:53:12,634 | 1.0.0 | INFO | ibmfl.party.party                               

After the message "Received Heartbeat from Aggregator" appears, then the Party is ready to start.

### 5.2 Start Training

In [14]:
p.start()

2020-12-08 20:53:12,643 | 1.0.0 | INFO | ibmfl.party.party                                  | Party not registered yet.
2020-12-08 20:53:12,645 | 1.0.0 | INFO | ibmfl.party.party                                  | Registering party...
2020-12-08 20:53:12,646 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | Sending serialized message to aggregator
2020-12-08 20:53:12,647 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | PartySendLoop: Number of active messages ready to send: 1
2020-12-08 20:53:13,209 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | Received Heartbeat from Aggregator
2020-12-08 20:53:14,608 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | PartySendLoop: Holding for message to send
2020-12-08 20:53:14,613 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | Received serialized message as response: <ibmfl.message.message.Message object at 0x7f90d042f990>
2020-12-08 20:53:14,

2020-12-08 20:53:55,576 | 1.0.0 | INFO | ibmfl.party.party_protocol_handler                 | Received request in PH 7
x_train shape: (500, 28, 28, 1)
500 train samples
500 test samples
2020-12-08 20:53:56,013 | 1.0.0 | INFO | ibmfl.party.training.local_training_handler        | Local model updated.
2020-12-08 20:53:56,014 | 1.0.0 | INFO | ibmfl.party.training.local_training_handler        | Local training started...
2020-12-08 20:53:56,016 | 1.0.0 | INFO | ibmfl.model.tensorflow_fl_model                    | Using default hyperparameters: epochs:1 batch_size:128
Train on 500 samples
500/500 [==============================] - 5s 9ms/sample - loss: 0.3675 - accuracy: 0.9000
2020-12-08 20:54:00,794 | 1.0.0 | INFO | ibmfl.party.training.local_training_handler        | Local training done, generating model update...
x_train shape: (500, 28, 28, 1)
500 train samples
500 test samples
500/500 [==============================] - 1s 2ms/sample - loss: 0.6037 - accuracy: 0.7880
2020-12-08 20:54:0

500/500 [==============================] - 1s 3ms/sample - loss: 0.5110 - accuracy: 0.8340
2020-12-08 20:54:35,885 | 1.0.0 | INFO | ibmfl.party.training.local_training_handler        | {'loss': 0.5110173730850219, 'acc': 0.83, 'accuracy': 0.8339999914169312}
2020-12-08 20:54:35,888 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | PartyRecvLoop : sent response
2020-12-08 20:54:37,898 | 1.0.0 | INFO | ibmfl.party.party_protocol_handler                 | Received request from aggregator
2020-12-08 20:54:37,899 | 1.0.0 | INFO | ibmfl.party.party_protocol_handler                 | Received request in with message_type:  14
2020-12-08 20:54:37,900 | 1.0.0 | INFO | ibmfl.party.party_protocol_handler                 | Received request in PH 14
2020-12-08 20:54:37,904 | 1.0.0 | INFO | ibmfl.party.party_protocol_handler                 | received a STOP request
2020-12-08 20:54:37,911 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | PartyRecvLoop : sent 

<a id = "summary"></a>
## Summary

Congratulations! You have learned to:

1. Start a Federated Learning experiment
2. Load a template model
3. Create an RTS and launch the experiment job
4. Load a dataset for training
5. Define the data handler
6. Configure the party
7. Connect to the aggregator
8. Train your Federated Learning model

### Learn more

- For more details about setting up Federated Learning, terminology, and running Federated Learning from the UI, see <a href = "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fed-lea.html?audience=wdp" target="_blank" rel="noopener no referrer">Federated Learning documentation</a> for Cloud.
- For more information on a Keras model template, see their documentation <a href = "https://www.tensorflow.org/tutorials/quickstart/advanced" target="_blank" rel="noopener no referrer">here</a>.

# <hr>
Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.
<br>
 
<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>